In [12]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pathlib
import cv2
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

# Load dataset CSVs
df_train = pd.read_csv("fungi_train.csv")
df_test = pd.read_csv("fungi_test.csv")

df_train["Path"] = df_train["Path"].apply(os.path.abspath)
df_test["Path"] = df_test["Path"].apply(os.path.abspath)

encoder = LabelEncoder()
df_train["ClassId"] = encoder.fit_transform(df_train["ClassId"])
num_classes = len(df_train["ClassId"].unique())

# Image properties
img_height, img_width = 180, 180
batch_size = 32

# Load and preprocess images efficiently
def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return np.zeros((img_height, img_width, 3), dtype=np.uint8)
    return cv2.resize(img, (img_height, img_width))

# Ensure 80/20 split for each class
train_images, val_images = [], []
train_labels, val_labels = [], []

for class_id in df_train["ClassId"].unique():
    class_subset = df_train[df_train["ClassId"] == class_id]
    
    # Ensure shuffling before splitting
    class_subset = class_subset.sample(frac=1, random_state=42).reset_index(drop=True)
    
    split_idx = int(0.8 * len(class_subset))  # 80% training, 20% validation
    
    train_images.extend(class_subset.iloc[:split_idx]["Path"].tolist())
    val_images.extend(class_subset.iloc[split_idx:]["Path"].tolist())

    train_labels.extend(class_subset.iloc[:split_idx]["ClassId"].tolist())
    val_labels.extend(class_subset.iloc[split_idx:]["ClassId"].tolist())

# Convert lists into arrays
X_train = np.array([load_and_preprocess_image(path) for path in tqdm(train_images, desc="Loading Training Data")])
X_val = np.array([load_and_preprocess_image(path) for path in tqdm(val_images, desc="Loading Validation Data")])

y_train = np.array(train_labels)
y_val = np.array(val_labels)

# Convert labels to categorical
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)

# Define ResNet50 model
resnet_model = keras.Sequential()
pretrained_model = tf.keras.applications.ResNet50(include_top=False,
                                                  input_shape=(img_height, img_width, 3),
                                                  pooling='avg',
                                                  weights='imagenet')

for layer in pretrained_model.layers:
    layer.trainable = False

resnet_model.add(pretrained_model)
resnet_model.add(keras.layers.Flatten())
resnet_model.add(keras.layers.Dense(512, activation='relu'))
resnet_model.add(keras.layers.Dense(num_classes, activation='softmax'))

resnet_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = resnet_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=batch_size, verbose=1)

# ORB Feature Extraction for Backup Matching
orb = cv2.ORB_create(nfeatures=30)
class_descriptors = {i: [] for i in df_train["ClassId"].unique()}

def extract_orb_features(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    return orb.detectAndCompute(img, None)[1]  # Return only descriptors

for row in tqdm(df_train.itertuples(), total=len(df_train), desc="Extracting ORB Features"):
    descriptors = extract_orb_features(row.Path)
    if descriptors is not None:
        class_descriptors[row.ClassId].append(descriptors)

def match_features(test_desc, train_desc_list):
    if test_desc is None:
        return 0
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    best_matches = 0
    
    for train_desc in train_desc_list:
        if train_desc is None:
            continue
        matches = matcher.knnMatch(test_desc, train_desc, k=2)
        good_matches = [m for match in matches if len(match) == 2 for m, n in [match] if m.distance < 0.7 * n.distance]
        best_matches = max(best_matches, len(good_matches))
    
    return best_matches

# Test Image Classification
predictions = []
for row in tqdm(df_test.itertuples(), total=len(df_test), desc="Classifying Test Images"):
    img = np.expand_dims(load_and_preprocess_image(row.Path), axis=0)
    cnn_pred = np.argmax(resnet_model.predict(img, verbose=0))
    
    test_desc = extract_orb_features(row.Path)
    match_scores = {class_id: match_features(test_desc, class_descriptors[class_id]) for class_id in class_descriptors}
    best_match = max(match_scores, key=match_scores.get)
    
    final_pred = cnn_pred if match_scores[best_match] < 10 else best_match
    predictions.append(final_pred)

# Save results
df_submission = pd.DataFrame({"id": df_test["id"], "output": predictions})
df_submission.to_csv("submission.csv", index=False)
print("Prediction complete. Results saved to submission.csv")

Loading Validation Data: 100%|██████████| 1000/1000 [00:00<00:00, 1558.08it/s]


Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 69s 522ms/step - accuracy: 0.7398 - loss: 0.7397 - val_accuracy: 0.8580 - val_loss: 0.2664
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 62s 500ms/step - accuracy: 0.8759 - loss: 0.2536 - val_accuracy: 0.8590 - val_loss: 0.2847
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 60s 477ms/step - accuracy: 0.8584 - loss: 0.3046 - val_accuracy: 0.9050 - val_loss: 0.2024
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 60s 479ms/step - accuracy: 0.9157 - loss: 0.2001 - val_accuracy: 0.8960 - val_loss: 0.2361
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 62s 496ms/step - accuracy: 0.8974 - loss: 0.2185 - val_accuracy: 0.9090 - val_loss: 0.2090
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 62s 494ms/step - accuracy: 0.9110 - loss: 0.1995 - val_accuracy: 0.9080 - val_loss: 0.2013
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 62s 495ms/step - accuracy: 0.9200 - loss: 0.1892 - val_accuracy: 0.8860 - val_loss: 0.2358
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 62s 498ms/step - accuracy: 0.9224 - loss: 0

Classifying Test Images: 100%|██████████| 1801/1801 [03:06<00:00,  9.65it/s]

Prediction complete. Results saved to submission.csv


In [22]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import cv2
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator

# Load dataset CSVs
df_train = pd.read_csv("fungi_train.csv")
df_test = pd.read_csv("fungi_test.csv")

df_train["Path"] = df_train["Path"].apply(os.path.abspath)
df_test["Path"] = df_test["Path"].apply(os.path.abspath)

# Encode labels
encoder = LabelEncoder()
df_train["ClassId"] = encoder.fit_transform(df_train["ClassId"])
num_classes = len(df_train["ClassId"].unique())

# Image properties
img_height, img_width = 180, 180
batch_size = 32

def load_and_preprocess_image(image_path):
    img = keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize
    return img_array

# Advanced Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3]
)

def contrast_stretch(img):
    """Apply contrast stretching using OpenCV."""
    min_val, max_val = np.min(img), np.max(img)
    img_stretched = (img - min_val) / (max_val - min_val + 1e-5)  # Normalize
    return np.clip(img_stretched, 0, 1)

def load_and_preprocess_image(image_path):
    img = keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = contrast_stretch(img_array)  # Apply contrast stretching
    img_array = img_array / 255.0  # Normalize
    return img_array

# Load all images and labels
train_images = df_train["Path"].tolist()
train_labels = df_train["ClassId"].tolist()
X_train = np.array([load_and_preprocess_image(path) for path in tqdm(train_images, desc="Loading Training Data")])
y_train = tf.keras.utils.to_categorical(train_labels, num_classes)

# Define VGG16 model with improvements
base_model = tf.keras.applications.VGG16(include_top=False, input_shape=(img_height, img_width, 3), pooling='avg', weights='imagenet')
for layer in base_model.layers[:-6]:  # Unfreeze last 6 layers for fine-tuning
    layer.trainable = False

model = keras.Sequential([
    base_model,
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# K-Fold Cross-Validation
kf = KFold(n_splits=3, shuffle=True, random_state=42)
fold_accuracies = []
for train_idx, val_idx in kf.split(X_train):
    X_tr, X_val = X_train[train_idx], X_train[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]
    
    history = model.fit(datagen.flow(X_tr, y_tr, batch_size=batch_size), validation_data=(X_val, y_val), epochs=10, verbose=1, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])
    
    fold_acc = max(history.history['val_accuracy'])
    fold_accuracies.append(fold_acc)

print(f'Average K-Fold Accuracy: {np.mean(fold_accuracies):.4f}')

# ORB Feature Extraction for Backup Matching
orb = cv2.ORB_create(nfeatures=150)
class_descriptors = {i: [] for i in df_train["ClassId"].unique()}

def extract_orb_features(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    keypoints, descriptors = orb.detectAndCompute(img, None)
    return descriptors

for row in tqdm(df_train.itertuples(), total=len(df_train), desc="Extracting ORB Features"):
    descriptors = extract_orb_features(row.Path)
    if descriptors is not None:
        class_descriptors[row.ClassId].append(descriptors)

# Test Image Classification
predictions = []
for row in tqdm(df_test.itertuples(), total=len(df_test), desc="Classifying Test Images"):
    img = np.expand_dims(load_and_preprocess_image(row.Path), axis=0)
    cnn_pred = np.argmax(model.predict(img, verbose=0))
    cnn_probs = model.predict(img, verbose=0)[0]
    
    test_desc = extract_orb_features(row.Path)
    match_scores = {class_id: max([len(cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True).match(test_desc, desc)) if desc is not None else 0 for desc in class_descriptors[class_id]]) for class_id in class_descriptors}
    best_match = max(match_scores, key=match_scores.get)
    
    # Adjust predictions to avoid class 4 bias
    if cnn_pred == 4 and (cnn_probs[4] < 0.5 or cnn_probs[0] > 0.3 or cnn_probs[1] > 0.3):
        final_pred = np.argmax([cnn_probs[0] + 0.1, cnn_probs[1] + 0.1, cnn_probs[2], cnn_probs[3], cnn_probs[4] - 0.1])
    else:
        final_pred = cnn_pred if match_scores[best_match] < 20 else best_match
    
    predictions.append(final_pred)

# Save results
df_submission = pd.DataFrame({"id": df_test["id"], "output": predictions})
df_submission.to_csv("submission.csv", index=False)
print("Prediction complete. Results saved to submission.csv")


Loading Training Data: 100%|██████████| 5000/5000 [00:04<00:00, 1057.92it/s]
c:\Documents\DATATHON\fungi-classification-shallow-learning\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 190s 2s/step - accuracy: 0.1974 - loss: 1.6094 - val_accuracy: 0.2088 - val_loss: 1.6605
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 196s 2s/step - accuracy: 0.2022 - loss: 1.6094 - val_accuracy: 0.2088 - val_loss: 1.6292
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 184s 2s/step - accuracy: 0.2024 - loss: 1.6094 - val_accuracy: 0.2088 - val_loss: 1.6200
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 183s 2s/step - accuracy: 0.2098 - loss: 1.6094 - val_accuracy: 0.2088 - val_loss: 1.6158
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 188s 2s/step - accuracy: 0.2023 - loss: 1.6094 - val_accuracy: 0.0000e+00 - val_loss: 1.6159
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 188s 2s/step - accuracy: 0.2066 - loss: 1.6094 - val_accuracy: 0.1980 - val_loss: 1.6153
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 189s 2s/step - accuracy: 0.2136 - loss: 1.6093 - val_accuracy: 0.1980 - val_loss: 1.6340
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 188s 2s/step - accuracy: 0.2103 - loss: 1.6094 - val_

Classifying Test Images: 100%|██████████| 1801/1801 [11:02<00:00,  2.72it/s]

Prediction complete. Results saved to submission.csv


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os
import cv2
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator
from keras.src.callbacks import ReduceLROnPlateau

# Enable XLA for faster execution
tf.config.optimizer.set_jit(True)

# Use mixed precision for faster training
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Load dataset CSVs
df_train = pd.read_csv("fungi_train.csv")
df_test = pd.read_csv("fungi_test.csv")

# Encode labels
encoder = LabelEncoder()
df_train["ClassId"] = encoder.fit_transform(df_train["ClassId"])
num_classes = len(df_train["ClassId"].unique())

# Image properties
img_height, img_width = 224, 224
batch_size = 32  # Reduced batch size for efficiency

# Image Preprocessing Function
def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img is None:
        return None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_width, img_height))
    return img / 255.0

# Data Augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

df_train["ClassId"] = df_train["ClassId"].astype(str)

# Generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    x_col="Path",
    y_col="ClassId",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

val_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    x_col="Path",
    y_col="ClassId",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

# Define EfficientNet model for improved efficiency
base_model = tf.keras.applications.EfficientNetB0(include_top=False, input_shape=(img_height, img_width, 3), pooling='avg', weights='imagenet')
base_model.trainable = False

model = keras.Sequential([
    base_model,
    keras.layers.Dense(256, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(num_classes, activation='softmax', dtype='float32')
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Dynamic Learning Rate Adjustment
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.5, verbose=1)

# Train with K-Fold Cross-Validation
kf = KFold(n_splits=3, shuffle=True, random_state=42)
fold_accuracies = []
for train_idx, val_idx in kf.split(df_train):
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=6,
        steps_per_epoch=len(train_generator) // 4,  # Reduce steps per epoch
        callbacks=[lr_scheduler],
        verbose=1
    )
    fold_acc = max(history.history['val_accuracy'])
    fold_accuracies.append(fold_acc)

print(f'Average K-Fold Accuracy: {np.mean(fold_accuracies):.4f}')

# Test Image Classification
predictions = []
for row in tqdm(df_test.itertuples(), total=len(df_test), desc="Classifying Test Images"):
    img = preprocess_image(row.Path)
    if img is None:
        predictions.append(-1)
        continue
    img = np.expand_dims(img, axis=0)
    cnn_pred = np.argmax(model.predict(img, verbose=0)[0])
    predictions.append(cnn_pred)

# Save results
df_submission = pd.DataFrame({"id": df_test["id"], "output": predictions})
df_submission.to_csv("submission.csv", index=False)
print("Prediction complete. Results saved to submission.csv")


Found 4000 validated image filenames belonging to 5 classes.
Found 1000 validated image filenames belonging to 5 classes.


c:\Documents\DATATHON\fungi-classification-shallow-learning\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/6
31/31 ━━━━━━━━━━━━━━━━━━━━ 66s 2s/step - accuracy: 0.4261 - loss: 1.5125 - val_accuracy: 0.0070 - val_loss: 2.1379 - learning_rate: 1.0000e-04
Epoch 2/6
31/31 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - accuracy: 0.6729 - loss: 0.7597 - val_accuracy: 0.0010 - val_loss: 2.2402 - learning_rate: 1.0000e-04
Epoch 3/6
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 947ms/step - accuracy: 0.6667 - loss: 0.7190
Epoch 3: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
31/31 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.6674 - loss: 0.7182 - val_accuracy: 0.0010 - val_loss: 2.3378 - learning_rate: 1.0000e-04
Epoch 4/6
31/31 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.7099 - loss: 0.6117 - val_accuracy: 0.0030 - val_loss: 2.4564 - learning_rate: 5.0000e-05
Epoch 5/6
 1/31 ━━━━━━━━━━━━━━━━━━━━ 27s 921ms/step - accuracy: 0.8125 - loss: 0.6446

c:\Documents\DATATHON\fungi-classification-shallow-learning\.venv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 5: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
31/31 ━━━━━━━━━━━━━━━━━━━━ 30s 982ms/step - accuracy: 0.8125 - loss: 0.6446 - val_accuracy: 0.0000e+00 - val_loss: 2.4782 - learning_rate: 5.0000e-05
Epoch 6/6
31/31 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.7301 - loss: 0.5820 - val_accuracy: 0.0020 - val_loss: 2.5973 - learning_rate: 2.5000e-05
Epoch 1/6
31/31 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.7148 - loss: 0.6305 - val_accuracy: 0.0030 - val_loss: 2.7397 - learning_rate: 2.5000e-05
Epoch 2/6
31/31 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - accuracy: 0.7022 - loss: 0.6122 - val_accuracy: 0.0010 - val_loss: 2.9356 - learning_rate: 2.5000e-05
Epoch 3/6
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 937ms/step - accuracy: 0.7339 - loss: 0.5971
Epoch 3: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-05.
31/31 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.7339 - loss: 0.5965 - val_accuracy: 0.0010 - val_loss: 3.0671 - learning_rate: 2.5000e-05
Epoch 4/6


Classifying Test Images: 100%|██████████| 1801/1801 [03:16<00:00,  9.18it/s]

Prediction complete. Results saved to submission.csv
